In [1]:
import pandas as pd 
import csv
import os
import matplotlib.pyplot as plt
import numpy as np

### Methodolgy
The Council for Community and Economic Research (C2ER) has annually published an unweighted average of prices accumulated from the previous three quarters since the beginning of quarter four of 2007. For the 2021 Cost of Living Index, they determined that the national average for 267 Urban Areas of the United States is COL Index 100, and they placed cities across the country in a list ranging from the most expensive to least expensive. Their Cost of Living Index measures regional differences in the cost of consumer goods and sercvices, excluding taxes and non-consumer expenditures. The index is based on six compounds: housing, utilities, grocery items, transportation, health care, and miscellaneous goods and services. However, this report would cost $275.00 per year for an electronic version, and that is not within the scope of this project.

Instead, in order to get the information we need, we will do our own calculations to find out Ohio's cost of living costs based on the values already found for Minnesota. The Cost of Living Calculator available at RentCafe.com (who gets their data from the C2ER Cost of Living Index) tells us that the cost of living for Minnesota is 1% lower than the national average, whereas Ohio is 9% lower than the national average.

![Minnesota COL](Datasets/Minnesota_COL_RentCafe.jpg)
![Ohio COL](Datasets/Ohio_COL_RentCafe.jpg)

In [12]:
csvpath = "Megan/Resources/mn_cost_of_living_2021.csv"

mn_col = pd.read_csv(csvpath)
mn_col.columns

Index(['Area', 'Family Size', 'Number of Adults', 'Number of Workers',
       'Number of Children', 'Age of Adults', 'Yearly Cost', 'Hourly Wage',
       'Child Care', 'Food', 'Health Care', 'Housing', 'Transport', 'Other',
       'Taxes'],
      dtype='object')

In [13]:
mn_col = mn_col[["Area", "Family Size", "Yearly Cost", "Hourly Wage", "Child Care", "Food", "Health Care", "Housing", "Transport", "Other", "Taxes"]]
mn_col.head()

,Area,Family Size,Yearly Cost,Hourly Wage,Child Care,Food,Health Care,Housing,Transport,Other,Taxes
0,Aitkin County,1,30874,14.84,0,341,191,570,863,249,359
1,Aitkin County,1,30471,14.65,0,353,152,570,863,253,348
2,Anoka County,1,36938,17.76,0,354,161,1066,680,389,428
3,Anoka County,1,37268,17.92,0,342,199,1066,680,385,434
4,Becker County,1,28659,13.78,0,353,152,592,722,259,310


In [18]:
# MN Average Family Size is 3.4 (rounded down to 3)
mn_avg = mn_col.mean()
mn_avg

Family Size        3.416667
Yearly Cost    54377.878115
Hourly Wage       20.259903
Child Care       414.593847
Food             932.501558
Health Care      512.090343
Housing          976.204050
Transport        794.956776
Other            522.539720
Taxes            378.607477
dtype: float64

In [38]:
# Minnesota does not list Utilities as a category in their dataset. Based on a search of several cost of living calculators, the difference in utilities between Minneapolis, MN
# Columbus, OH is 2-4% in favor of OH

n_avg_food = mn_avg["Food"] - (mn_avg["Food"]*0.03)
print(f"National Average (Food): {n_avg_food}")
n_avg_health = mn_avg["Health Care"] - (mn_avg["Health Care"]*0.06)
print(f"National Average (Health Care): {n_avg_health}")
n_avg_housing = mn_avg["Housing"] + (mn_avg["Housing"]*0.11)
print(f"National Average (Housing): {n_avg_housing}")
n_avg_transport = mn_avg["Transport"] + (mn_avg["Transport"]*0.02)
print(f"National Average (Transport): {n_avg_transport}")
n_avg_other = mn_avg["Other"] - (mn_avg["Other"]*0.07)
print(f"National Average (Other): {n_avg_other}")

national_averages = pd.DataFrame(
    {"Type": ["Food", "Health Care", "Housing", "Transport", "Other"],
    "National Average": [n_avg_food, n_avg_health, n_avg_housing, n_avg_transport, n_avg_other]})
national_averages.set_index("Type")
national_averages


National Average (Food): 904.5265109034268
National Average (Health Care): 481.3649221183801
National Average (Housing): 1083.586495327103
National Average (Transport): 810.8559112149533
National Average (Other): 485.96193925233644


,Type,National Average
0,Food,904.526511
1,Health Care,481.364922
2,Housing,1083.586495
3,Transport,810.855911
4,Other,485.961939


In [24]:
almost_oh = mn_avg[["Food", "Health Care", "Housing", "Transport", "Other"]]
almost_oh

Food           932.501558
Health Care    512.090343
Housing        976.204050
Transport      794.956776
Other          522.539720
dtype: float64

In [72]:
value_list = []
for item in almost_oh:
    value_list.append(item)

print(value_list)

[932.5015576323988, 512.0903426791277, 976.2040498442368, 794.9567757009346, 522.5397196261682]


In [75]:
# mn_nums = almost_oh.to_frame()
# mn_renamed = mn_nums.rename(index={0:"Type"}, columns={"0": "Minnesota"}, inplace=True)
# mn_renamed.head()
mn_dict = {"Type": ["Food", "Health Care", "Housing", "Transport", "Other"], "MN": value_list}
mn_nums = pd.DataFrame(mn_dict)
mn_nums

,Type,MN
0,Food,932.501558
1,Health Care,512.090343
2,Housing,976.204050
3,Transport,794.956776
4,Other,522.539720


In [76]:
mn_national = pd.merge(national_averages, mn_nums, on="Type")
mn_national

,Type,National Average,MN
0,Food,904.526511,932.501558
1,Health Care,481.364922,512.090343
2,Housing,1083.586495,976.204050
3,Transport,810.855911,794.956776
4,Other,485.961939,522.539720


In [79]:
# Ohio amounts
oh_food = (n_avg_food - (n_avg_food)*0.01)
print(f"Ohio (Food): {oh_food}")
oh_health = (n_avg_health - (n_avg_health)*0.03)
print(f"Ohio (Health Care): {oh_health}")
oh_housing = (n_avg_housing - (n_avg_housing)*0.23)
print(f"Ohio (Housing): {oh_housing}")
oh_transport = (n_avg_transport - (n_avg_transport)*0.02)
print(f"Ohio (Transport): {oh_transport}")
oh_other = (n_avg_other - (n_avg_other)*0.01)
print(f"Ohio (Other): {oh_other}")

ohio_df = pd.DataFrame(
    {"Type": ["Food", "Health Care", "Housing", "Transport", "Other"],
    "Ohio": [oh_food, oh_health, oh_housing, oh_transport, oh_other]})
ohio_df

Ohio (Food): 895.4812457943925
Ohio (Health Care): 466.9239744548287
Ohio (Housing): 834.3616014018693
Ohio (Transport): 794.6387929906542
Ohio (Other): 481.10231985981306


,Type,Ohio
0,Food,895.481246
1,Health Care,466.923974
2,Housing,834.361601
3,Transport,794.638793
4,Other,481.102320


In [80]:
ohio_mn_compare = pd.merge(mn_national, ohio_df, on="Type")
ohio_mn_compare

,Type,National Average,MN,Ohio
0,Food,904.526511,932.501558,895.481246
1,Health Care,481.364922,512.090343,466.923974
2,Housing,1083.586495,976.204050,834.361601
3,Transport,810.855911,794.956776,794.638793
4,Other,485.961939,522.539720,481.102320


In [81]:
ohio_mn_compare.to_csv("ohio_mn_col_compare.csv", encoding="utf-8", index=False)